In [1]:
import uproot as up
import matplotlib as plot
import plotly.graph_objects as go
import plotly
import numpy as np

In [17]:
# Load the ROOT file
event_idx = 0

file = up.open("/hpc/group/vossenlab/rck32/eic/work_eic/root_files/Clustering/February_19/_Feb_19_pip_50events_run_1_50_142.edm4hep.root") #pion
# file = up.open("/hpc/group/vossenlab/rck32/eic/work_eic/root_files/Clustering/February_19/Feb_19_mum_50events_run_1_50_1.edm4hep.root") #muon
# file = up.open("/hpc/group/vossenlab/rck32/eic/work_eic/root_files/Clustering/January_26/jan_26_50events_50_0.edm4hep.root")
# file = up.open("/hpc/group/vossenlab/rck32/eic/work_eic/root_files/Clustering/February_14/test_Feb_13_50events_run_1_numsims_200_50_217.edm4hep.root")
tree = file["events"]  # Replace with actual tree name

# Extract particle vertex positions
vertices = tree.arrays(["MCParticles.vertex.x", "MCParticles.vertex.y", "MCParticles.vertex.z", "_MCParticles_parents.index","MCParticles.parents_begin","MCParticles.parents_end"], library="np")

x, y, z, parent_id,parent_begin,parent_end = vertices["MCParticles.vertex.x"][event_idx], vertices["MCParticles.vertex.y"][event_idx], vertices["MCParticles.vertex.z"][event_idx], vertices["_MCParticles_parents.index"][event_idx],vertices["MCParticles.parents_begin"][event_idx],vertices["MCParticles.parents_end"][event_idx]

x_endpoint = tree["MCParticles.endpoint.x"].array(library = "np")[event_idx]
y_endpoint = tree["MCParticles.endpoint.y"].array(library = "np")[event_idx]
z_endpoint = tree["MCParticles.endpoint.z"].array(library = "np")[event_idx]
simulatorStatus = tree["MCParticles.simulatorStatus"].array(library = "np")[event_idx]
PDG = tree["MCParticles.PDG"].array(library = "np")[event_idx]

## Basic cube detector

In [18]:
# Define the detector as a wireframe cube (Modify for real detector shape)
detector_edges = [
    ([0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 0, 0]),  # Bottom square
    ([0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [1, 1, 1, 1, 1]),  # Top square
    ([0, 0], [0, 0], [0, 1]),  # Vertical edges
    ([1, 1], [0, 0], [0, 1]),
    ([1, 1], [1, 1], [0, 1]),
    ([0, 0], [1, 1], [0, 1]),
]

detector_traces = [
    go.Scatter3d(
        x=edge[0], y=edge[1], z=edge[2],
        mode="lines",
        line=dict(color="blue", width=2),
        name="Detector"
    ) for edge in detector_edges
]


# KLM Inner outline

In [19]:


# Detector parameters
r = 1770  # Inner radius in mm
z_top = 750
z_bottom = -750

# Define 8-sided polygon (octagon) in XY plane
angles = np.linspace(0, 2 * np.pi, 9)[:-1]  # 8 points, last point omitted to avoid duplicate
x_vertices = r * np.cos(angles)
y_vertices = r * np.sin(angles)

# Create edges of the octagonal barrel
detector_edges = []

# Connect top and bottom octagon edges
for i in range(8):
    # Top face edges
    detector_edges.append(([x_vertices[i], x_vertices[(i+1) % 8]],
                           [y_vertices[i], y_vertices[(i+1) % 8]],
                           [z_top, z_top]))
    
    # Bottom face edges
    detector_edges.append(([x_vertices[i], x_vertices[(i+1) % 8]],
                           [y_vertices[i], y_vertices[(i+1) % 8]],
                           [z_bottom, z_bottom]))
    
    # Vertical edges connecting top to bottom
    detector_edges.append(([x_vertices[i], x_vertices[i]],
                           [y_vertices[i], y_vertices[i]],
                           [z_top, z_bottom]))

# Convert to Plotly traces
detector_traces = [
    go.Scatter3d(
        x=edge[0], y=edge[1], z=edge[2],
        mode="lines",
        line=dict(color="blue", width=2),
        name="Detector"
    ) for edge in detector_edges
]


## KLM Full barrel

In [20]:

# Inner and outer radii
r_inner = 1770  # mm
r_outer = r_inner + 1060  # mm
z_top = 750
z_bottom = -750

# Define octagon vertices for both inner and outer surfaces
angles = np.linspace(0, 2 * np.pi, 9)[:-1]  # 8 points

x_inner = r_inner * np.cos(angles)
y_inner = r_inner * np.sin(angles)

x_outer = r_outer * np.cos(angles)
y_outer = r_outer * np.sin(angles)

# List to store detector edges
detector_edges = []

# Create edges for both the inner and outer octagons
for i in range(8):
    # Inner octagon (top and bottom)
    detector_edges.append(([x_inner[i], x_inner[(i+1) % 8]],
                           [y_inner[i], y_inner[(i+1) % 8]],
                           [z_top, z_top]))  # Top face
    detector_edges.append(([x_inner[i], x_inner[(i+1) % 8]],
                           [y_inner[i], y_inner[(i+1) % 8]],
                           [z_bottom, z_bottom]))  # Bottom face

    # Outer octagon (top and bottom)
    detector_edges.append(([x_outer[i], x_outer[(i+1) % 8]],
                           [y_outer[i], y_outer[(i+1) % 8]],
                           [z_top, z_top]))  # Top face
    detector_edges.append(([x_outer[i], x_outer[(i+1) % 8]],
                           [y_outer[i], y_outer[(i+1) % 8]],
                           [z_bottom, z_bottom]))  # Bottom face

    # Connect top to bottom for inner and outer surfaces
    detector_edges.append(([x_inner[i], x_inner[i]],
                           [y_inner[i], y_inner[i]],
                           [z_top, z_bottom]))  # Inner vertical edges
    detector_edges.append(([x_outer[i], x_outer[i]],
                           [y_outer[i], y_outer[i]],
                           [z_top, z_bottom]))  # Outer vertical edges

    # Connect inner to outer at top and bottom
    detector_edges.append(([x_inner[i], x_outer[i]],
                           [y_inner[i], y_outer[i]],
                           [z_top, z_top]))  # Top radial edges
    detector_edges.append(([x_inner[i], x_outer[i]],
                           [y_inner[i], y_outer[i]],
                           [z_bottom, z_bottom]))  # Bottom radial edges

# Convert edges to Plotly traces
detector_traces = [
    go.Scatter3d(
        x=edge[0], y=edge[1], z=edge[2],
        mode="lines",
        line=dict(color="blue", width=2),
        name="Detector"
    ) for edge in detector_edges
]


In [21]:
# Plot the scatterpoints for the vertices
colors = ["blue" for point in range(len(x))]
colors[0] = "red"
particle_trace = go.Scatter3d(
    x=x, y=y, z=z,
    mode="markers",
    marker=dict(size=1, color=colors),
#     marker_symbol="x",
    name="Particles"
)

#Plot the primary endpoint
# endpoint_trace = go.Scatter3d(
#     x = [x_endpoint[0]], y=[y_endpoint[0]], z=[z_endpoint[0]],
#     mode="markers",
#     marker_symbol = "x",
#     marker = dict(size=5, color = "red")
# )

In [22]:
line_traces = []
for i in range(len(x)):
    real_parent = parent_id[parent_begin[i]]
    if parent_end[i] - parent_begin[i] > 0:  # Assuming -1 means no parent
        px, py, pz = x[real_parent], y[real_parent], z[real_parent]
        line_traces.append(
            go.Scatter3d(
                x=[px, x[i]], y=[py, y[i]], z=[pz, z[i]],
                mode="lines",
                line=dict(color="gray", width=1),
                name="Parent-Child Connection",
                showlegend=False
            )
        )


In [23]:
fig = go.Figure(data=[particle_trace] + detector_traces + line_traces)
# fig = go.Figure(data=[particle_trace] + detector_traces + line_traces + [endpoint_trace])
fig.update_layout(
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z",
        aspectmode="data"
    ),
    title="3D Event Display"
)
fig.show()
fig.write_html("event_display_pip.html")
